# 교차 검증과 그리드 서치
## 검증 세트

- 앞에서는 훈련 세트(80%) : 테스트 세트(20%) 비율로 나눴음.

- 검증세트는 훈련세트에서 추가로 20% 정도를 떼어냄.

- 검증세트는 테스트 세트를 사용하지 않고 모델이 과대적합인지 과소적합인지 파악하기 위해 사용.

- 훈련 세트에서 모델을 훈련 => 검증 세트로 모델을 평가

- 이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고름

- 그 다음 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련

- 마지막에 테스트 세트에서 최종 점수를 평가

In [1]:
# 데이터 불러오기
import pandas as pd
wine = pd.read_csv('./db/wine_csv_data.csv')

In [2]:
# class 열을 타깃으로 사용하고 나머지 열은 특성 배열에 저장
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
# 훈련 세트와 테스트 세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
# train_input & train_target 을 다시 훈련 세트와 검증세트로 나누기.
# train_input 의 20%를 val_input 으로 만들기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


- 훈련 세트에 과대적합 되어있음 => 매개변수를 바꿔서 더 좋은 모델을 찾아야 함.

## 교차 검증(cross validation)